In [2]:
# 1.04.2024
# добавлены первые описания блоков

На картинке изображена структурная схема модели, которую планируется реализовать.

- Желтым цветом отмечены блоки, функции для которых на данный момент прописаны.

- Синим цветом отмечены блоки, над которыми либо ведется работа, либо они просто пропущены (тк не написан другой блок, без которого этот не может обойтись), но будут в финальной модели.

- Красным цветом отмечены блоки, которые делать не планируется.

![Структурная схема модели](model_structure.png)

Каждый блок этой диаграммы (или набор блоков) реализован как отдельная функция, преобразующая входной в нее массив.

Основной управляющий файл `ofdm_modem.m` сохраняет данные с выхода каждого блока в txt файл. Поэтому можно смотреть как преобразуется сигнал от блока к блоку.

На вход он принимает следующие параметры - порядок модулятора `M` (2,4,8,16,64...); длину фрейма `fr_len` (степень двойки); соотношение сигнал/шум на приемнике в децибелах `SNR_dB`; длину циклического префикса `cp_length`

# Передатчик

Включает все бллоки до передачи сигнала в канал

## Блок Bits stream

Генерирует случайную последовательность чисел от 0 до `M-1` длины такой же как длина фрейма. Используется функция `randi()`

## Блок Modulator

Переводит числовую последовательность в последовательность комплексных чисел из созвездия модулятора.
Реализован функцией `generate_information_frame(message, M)`
Внутри функции:
- Если `M` равно 2, 4, 8 - то используется M-PSK модулятор, реализованный в функции `pskmod()`
- Если `M` Больше или равно 16 - то используется QAM модулятор, реализованный в функции `qammod()`

## Блок "S/P -> IFFT -> P/S"

Переводит сигнал из частотной во временную область.

В модели реализован с помощью встроенной функции `IFFT`

## Блок Add cyclic prefix

Реализлован функцией `add_cyclic_prefix(frame_td, l)`
Берет последние `l` символов из конца массива отсчетов `frame_td` сигнала во временной области и добавляет их в начало. Важно, чтоб `l` была меньше длины массива (обычно равна половине длины).

![conv_feature](svertka.png)

# Канал

## Блок Channel

В разработке

## Блок AWGN

Добавляет белый гауссовский шум к сигналу, прошедшему через канал. С помощью функции `awgn()`. Дисперсия этого шума определяется значением `SNR_dB`

# Приемник

## Блок Remove cyclic prefix

Реализлован функцией `remove_cyclic_prefix(frame_td, l)`
Удаляет первые `l` отсчетов из конца массива отсчетов `frame_td` сигнала во временной области на приемнике. Важно, чтоб `l` была меньше длины массива (обычно равна половине длины).

## Блок "S/P -> IFFT"

Переводит сигнал из временной обратно в частотную область.

В модели реализован с помощью встроенной функции `FFT`.

## Блок Remove pilot signals

Еще не реализован. Должен будет отделять пилотные сигналы и отправлять их в эквалайзер для оценки канала.

## Блок Equalizer

Еще не реализован. Должен будет делать оценку канала по пилотным сигналам, а также по этой оценке преобразовывать принятый информационный сигнал.

## Блок Demodulator

Переводит последовательность комплексных чисел из созвездия модулятора в последовательность целых чисел (или бинарную последовательность)
Реализован функцией `decode_frame(frame, M)`
Внутри функции:
- Если `M` равно 2, 4, 8 - то используется M-PSK демодулятор, реализованный в функции `pskdemod()` - значения всех параметров наследуются из блока <b>Modulator</b>
- Если `M` Больше или равно 16 - то используется QAM демодулятор, реализованный в функции `qamdemod()` - значения всех параметров наследуются из блока <b>Modulator</b>

## Блок EVM

Считает метрику <b>Error Vector Magnitude</b> с помощью встренной функции `lteEVM` и с помощью моей функции `evaluate_evm()`. На вход обе принимают массив символов (еще не демодулированных) с приемника $S_n(\text{Rx})$ и исходную модулированную последовательность с модулятора передатчика $S_n(\text{Tx})$.

Используется формула:

$$\text{EVM}=\sqrt{\dfrac{\displaystyle{\sum_{n=1}^N}\big|S_n(\text{Tx})-S_n(\text{Rx})\big|^2}{N\cdot P_s(\text{Tx})}}$$

где $N$ - число символов в фрейме, $P_s(\text{Tx}) = \dfrac{1}{N}\sum_{n=1}^N |S_n(\text{Tx})|^2$ - средняя мощность источника

## Блок Decoded bits stream -> BER

Считает метрику <b>Uncoded BER</b> с помощью встренной функции `biterr` и с помощью моей функции `evaluate_ber()`. На вход обе принимают демодулированное сообщение с приемника и исходное сообщение с передатчика. Переводят полученные целые числа в последовательности бит. Считают суммарное количество неправильных бит и делят на общее количество принятых бит.